## 0. Setup + dataset (Adult Income)

In [12]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, cross_val_score

# Préprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# Outils pipelines
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Metrics et évaluation
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [13]:
# 1) Chargement dataset réel : Adult (Census Income)
adult = fetch_openml("adult", version=2, as_frame=True)
X = adult.data
y = (adult.target == ">50K").astype(int)  # binaire 0/1

# 2) Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 3) Colonnes num / cat
cat_cols = X.select_dtypes(include=["object", "category", "string"]).columns
num_cols = X.columns.difference(cat_cols)

print("Num cols:", len(num_cols), "| Cat cols:", len(cat_cols))

Num cols: 6 | Cat cols: 8


## 1. Préprocessing manuel (hors pipeline) + modèle “hors pipeline”

### 1.1 Imputer + scaler (num) / imputer + one-hot (cat), à la main

In [14]:
# --- NUM : imputation + scaling (fit sur train, transform train/test)
num_imputer = SimpleImputer(strategy="median")
num_scaler = StandardScaler()

X_train_num = num_imputer.fit_transform(X_train[num_cols])
X_test_num  = num_imputer.transform(X_test[num_cols])

X_train_num = num_scaler.fit_transform(X_train_num)
X_test_num  = num_scaler.transform(X_test_num)


In [15]:
# --- CAT : imputation + one-hot
cat_imputer = SimpleImputer(strategy="most_frequent")
ohe = OneHotEncoder(handle_unknown="ignore")

X_train_cat = cat_imputer.fit_transform(X_train[cat_cols])
X_test_cat  = cat_imputer.transform(X_test[cat_cols])

X_train_cat = ohe.fit_transform(X_train_cat)
X_test_cat  = ohe.transform(X_test_cat)


In [16]:
# --- CONCAT (attention: sparse)
from scipy.sparse import hstack

X_train_prepared = hstack([X_train_num, X_train_cat])
X_test_prepared  = hstack([X_test_num,  X_test_cat])

print("Shape after preprocessing:", X_train_prepared.shape)

Shape after preprocessing: (39073, 105)


### 1.2 Modèle hors pipeline

In [18]:
clf = LogisticRegression(max_iter=5000, solver="saga")
clf.fit(X_train_prepared, y_train)

y_pred = clf.predict(X_test_prepared)
print("Accuracy (manual preprocessing):", accuracy_score(y_test, y_pred))

Accuracy (manual preprocessing): 0.8521854846964889


**Remarques :**

- “On doit se souvenir d’appliquer exactement les mêmes transfos au test”

- “On doit faire fit uniquement sur train”

- “C’est vite ingérable quand on fait de la CV / GridSearch”

## 2. L’état de l’art : Pipeline, make_pipeline, ColumnTransformer

### 2.1 ColumnTransformer = préprocessing “par type de colonne”

In [ ]:
# -- Préprocessing dans un pipeline -- #

numeric_preprocess = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_preprocess = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer([
    ("num", numeric_preprocess, num_cols),
    ("cat", categorical_preprocess, cat_cols),
])

> **Message clé :ColumnTransformer applique en parallèle des étapes différentes selon les colonnes.**

In [20]:
# -- application du preprocessing via le pipeline -- #
X_train_prepared = preprocess.fit_transform(X_train)
X_test_prepared = preprocess.transform(X_test)

# Modele et évaluation (même code que précédemment)
clf = LogisticRegression(max_iter=5000, solver="saga")
clf.fit(X_train_prepared, y_train)

y_pred = clf.predict(X_test_prepared)
print("Accuracy (pipeline preprocessing):", accuracy_score(y_test, y_pred))

Accuracy (pipeline preprocessing): 0.8521854846964889


### 2.2 Pipeline (version explicite, nommée)

In [21]:
# -- Création d'un pipeline complet : preprocessing + modèle -- # 

pipe_lr = Pipeline([
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=5000, solver="saga"))
])

# -- Entraînement du pipeline (fit sur train, transform + predict/score sur test) -- #
pipe_lr.fit(X_train, y_train)
print("Accuracy (Pipeline):", pipe_lr.score(X_test, y_test))

Accuracy (Pipeline): 0.8521854846964889


> **Message clé : là, tu donnes au modèle les données brutes → le pipeline fait tout.**

### 2.3 make_pipeline (raccourci)
Même chose mais sans nommer toi-même les étapes :

In [ ]:
# -- Même pipeline mais avec make_pipeline (pas de noms d'étapes, les noms sont générés automatiquement) -- #
pipe_lr2 = make_pipeline(
    preprocess,
    LogisticRegression(max_iter=5000, solver="saga")
)

pipe_lr2.fit(X_train, y_train)
print("Accuracy (make_pipeline):", pipe_lr2.score(X_test, y_test))

Accuracy (make_pipeline): 0.8521854846964889


> **Message clé : make_pipeline = plus rapide, mais noms auto (pratique, moins lisible en GridSearch).**

> ### Best practice

 - Toujours mettre preprocessing dans un Pipeline

 - Utiliser ColumnTransformer pour données mixtes

 - Faire tuning / CV sur le pipeline, jamais sur données déjà transformées

### 2.4 Visualisation de pipeline (optionnelle, pour comprendre ce qui se passe dans le pipeline)

In [ ]:
# -- Visualisation des différents pipelines -- #
from sklearn import set_config
set_config(display="diagram")  # pour afficher les pipelines sous forme de diagramme

print("Preprocessing pipeline:")
display(preprocess)
print(f"\n{'*'*60}\n")

print("Pipeline avec preprocessing + modèle (Pipeline):")
display(pipe_lr)
print(f"\n{'*'*60}\n")

print("Pipeline avec preprocessing + modèle (make_pipeline):")
display(pipe_lr2)

Preprocessing pipeline:


,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `


************************************************************

Pipeline avec preprocessing + modèle (Pipeline):


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont


************************************************************

Pipeline avec preprocessing + modèle (make_pipeline):


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('logisticregression', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the differe

## 3. Modèle dans le pipeline + plusieurs pipelines (modèles + preprocessing)

### Vrai bénéfice : 
> **Il est possible de comparer plusieurs modèles facilement, et chacun peut avoir un preprocessing adapté.**

Pour faire la comparaison de plusieurs modèles avec leurs propres préprocessings, on peut faire un dictionnaire de pipelines, et ensuite faire une boucle pour les entraîner et les évaluer.

### 3.1 Créer plusieurs pipelines

In [ ]:
pipelines = {
    "LogReg (scaled + OHE)": Pipeline([
        ("preprocess", preprocess),
        ("model", LogisticRegression(max_iter=5000, solver="saga"))
    ]),
    
    # KNN = distance => scaling important (et OHE pour cat)
    "KNN (scaled + OHE)": Pipeline([
        ("preprocess", preprocess),
        ("model", KNeighborsClassifier(n_neighbors=15))
    ])
}

# Idée d'autre pipelines à ajouter :
# RandomForest = arbres => scaling pas nécessaire, mais OHE utile ici (sklearn)
# On peut faire un preprocess "sans scaler" pour illustrer

preprocess_no_scaler = ColumnTransformer([
    ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols),
    ("cat", categorical_preprocess, cat_cols),
])

pipelines["RandomForest (no scale + OHE)"] = Pipeline([
    ("preprocess", preprocess_no_scaler),
    ("model", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))
])

In [44]:
# Visualisation du dictionnaire de pipelines
list(pipelines.keys())

['LogReg (scaled + OHE)',
 'KNN (scaled + OHE)',
 'RandomForest (no scale + OHE)']

### 3.2 Comparaison propre en cross-validation (recommandé)

In [45]:
for name, pipe in pipelines.items():
    scores = cross_val_score(pipe, X_train, y_train, cv=5, scoring="accuracy")
    print(f"{name:30s} | acc CV: {scores.mean():.3f} ± {scores.std():.3f}")

LogReg (scaled + OHE)          | acc CV: 0.851 ± 0.004
KNN (scaled + OHE)             | acc CV: 0.841 ± 0.004
RandomForest (no scale + OHE)  | acc CV: 0.852 ± 0.002


> #### Point clé :
- Comparaison des pipelines complets (donc sans fuite)
- c’est plug-and-play pour GridSearchCV

## Pour aller plus loin :

### Gestion de pipelines dans un projet ML (structure de dossier, etc..)

**Commande pour créer une structure de dossier toute simple pour un projet ML avec des pipelines :**  

```sh
mkdir -p src notebooks data models
touch src/pipelines.py src/data_preprocessing.py src/model.py src/utils.py src/main.py
```

**Explication de la structure de dossier proposée :**

- src/ : contient l’ensemble des scripts de preprocessing, modèles, utils, etc.

    - pipelines.py : contient les définitions des pipelines (preprocessing + modèle)

    - data_preprocessing.py : contient les fonctions de preprocessing (imputation, scaling, encoding, etc.)

    - model.py : contient les définitions des modèles (RandomForest, LogisticRegression, etc.)

    - utils.py : contient les fonctions utilitaires (évaluation, visualisation, etc.).

    - main.py : script principal pour entraîner les modèles, faire les prédictions, etc.

- notebooks/ : contient les notebooks de développement, exploration, etc.

- data/ : contient les données brutes et éventuellement les données transformées.

- models/ : contient les modèles entraînés (pickle, joblib, etc.).

**Exemple de contenu des fichiers .py**

In [ ]:
##-- data_preprocessing.py --##

def preprocess_data(X_train, X_test, num_cols, cat_cols):
    # ... code de preprocessing vu précédemment (imputation, scaling, encoding) ...
    return X_train_prepared, X_test_prepared

In [ ]:
##-- model.py --##

def train_model(X_train, y_train):
    # ... code d'entraînement du modèle (ex: LogisticRegression, RandomForest, etc.) ...
    return model


def evaluate_model(model, X_test, y_test):
    # ... code d'évaluation du modèle (accuracy, classification report, etc.) ...
    return metrics

In [ ]:
##-- pipelines.py --##
from data_preprocessing import preprocess_data

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

def create_pipeline(num_cols, cat_cols):
    # ... code de création de pipelines (preprocessing + modèle) ...
    return pipeline

In [ ]:
##-- main.py --##
from pipelines import create_pipeline
from data_preprocessing import preprocess_data
from model import train_model, evaluate_model


def main():
    # 1) Chargement des données
    # 2) Split train/test
    # 3) Création des pipelines
    # 4) Entraînement et évaluation des pipelines
    pass